## OpenVisus Enabled Jupyter Notebook

### OpenViSUS: imports and utilities

In [2]:
%matplotlib notebook

import os,sys

# Here are commands to install a package (OpenVisus) directly from a jupyter notebook
# after you install those once you can comment those comment
# !{sys.executable} -m pip install OpenVisus

import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *

import OpenVisus as ov

# Enable I/O component of OpenVisus
ov.DbModule.attach()

PythonEngine is working fine


In [3]:
# function to read data from a remote dataset
# optional parameters: timestep, field (variable in the dataset), logic_box (bounding box of the query), resolution

# Note: the resolution value could sometime fetch a dataset with the wrong aspect ratio, 
# this because in the IDX format we double the size at each resolution on only one of the axis at a time

def readData(dataset,
    time=None,
    field=None,
    logic_box=ov.BoxNi(),
    resolution=None):
    
    # setting default values if not provided
    if(time==None):
        time=dataset.getDefaultTime()
    if(field==None):
        field=dataset.getDefaultField()
    if(logic_box==ov.BoxNi()):
        logic_box=dataset.getLogicBox()
    if(resolution==None):
        resolution=dataset.getMaxResolution()
    
    # create an access mode (default "network") to access the data
    access=dataset.createAccess()
    # define a box query to fetch data from a certain dataset, field and timestep
    query=ov.BoxQuery(dataset, field, time,ord('r'))
    # set the bounding box for our query
    query.logic_box=logic_box
    # set the resolution 
    query.end_resolutions.push_back(resolution)
    # prepare and execute the query
    dataset.beginQuery(query)
    dataset.executeQuery(access,query)

    # transform the result of the query to a numpy array
    data=ov.Array.toNumPy(query.buffer,bSqueeze=True,bShareMem=False)
    
    return data

# function to plot the image data with matplotlib
# optional parameters: colormap, existing plot to reuse (for more interactivity)
def showData(data, cmap=None, plot=None):
    if(plot==None or cmap!=None):
        fig=plt.figure(figsize = (7,5))
        plot = plt.imshow(data, origin='lower', cmap=cmap)
        plt.show()
        return plot
    else:
        plot.set_data(data)
        plt.show()
        return plot
    